<a href="https://colab.research.google.com/github/NahuelRepetto/Programacion-Concurrente/blob/main/TrabajoN1Parte2Java.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
%%writefile Matrices.java
import java.io.IOException;
import java.util.ArrayList;

public class Matriz
{  
  private static final int MAX_DIMENSION = 20;
  private static final int MIN_DIMENSION = 5;
  private static final int MAX_VALOR = 32;
  private static final int MIN_VALOR = -32;

  public static class Hilo extends Thread
  {
    private int dimension;
    private int fila;
    private int[][] matrizA;
    private int[][] matrizB;
    private int[][] matrizC;
    
    public Hilo(int dimension, int fila, int[][] matrizA, int[][] matrizB, int[][] matrizC)
    {
      super();
      this.dimension = dimension;
      this.fila = fila;
      this.matrizA = matrizA;
      this.matrizB = matrizB;
      this.matrizC = matrizC;
    }

    public void run()
    {      
      for(int i=0; i<dimension; i++)
        for(int j=0; j<dimension; j++)
          matrizC[fila][i] += matrizA[fila][j] * matrizB[j][i];      
    }
  }

  public static int[][] inicializarMatriz(int dimension, int min, int max)
  {
    int [][] matriz = new int [dimension][dimension];
    for(int i=0; i<dimension; i++)
    {
      for(int j=0; j<dimension; j++)          
      matriz[i][j] = (int)(Math.random()*(max-min+1)+min);       
    } 
    return matriz;
  }

  public static void calcularProducto(int dimension, int fila, int[][] matrizA, int[][] matrizB, int[][] matrizC)
  {
    for(int i=0; i<dimension; i++)
      for(int j=0; j<dimension; j++)
        matrizC[fila][i] += matrizA[fila][j] * matrizB[j][i];  
  }

  public static void mostrarMatriz(int dimension, int[][] matriz, String nombre)
  {  
    System.out.println();  
    System.out.println("Matriz " + nombre);
    for(int i = 0; i < dimension; i++) 
    {
      System.out.print("| ");
      for (int j = 0; j < dimension; j++) 
      {
        if(matriz[i][j] < -999)
          System.out.print(matriz[i][j] + " ");
        else if(matriz[i][j] < -99 || matriz[i][j] > 999)
          System.out.print(" " + matriz[i][j] + " ");             
        else if(matriz[i][j] < -9 || matriz[i][j] > 99)
          System.out.print("  " + matriz[i][j] + " ");          
        else if(matriz[i][j] < 0 || matriz[i][j] > 9)
          System.out.print("   " + matriz[i][j] + " ");          
        else if(matriz[i][j] < 10)
          System.out.print("    " + matriz[i][j] + " ");            
      }
      System.out.print("  |");
      System.out.println();
    }  
  }

  public static void main(String[] args) throws InterruptedException 
  {    
    //VERIFICO QUE SE HAYA ENVIADO EL NUMERO DE FILAS/COLUMNAS
    if( args.length < 1 ) 
    {
      System.out.println("Debe ingresar el numero de filas/columnas, dentro del rango de " + MIN_DIMENSION + " a " + MAX_DIMENSION); 
      System.exit(1);
    }

    int n = Integer.parseInt(args[0]); //DIMENSIN DE LA MATRIZ     

    //VERIFICO QUE SE HAYA ENVIADO EL NUMERO DENTRO DE LOS LIMITES ESTABLECIDOS
    if( n < MIN_DIMENSION || n > MAX_DIMENSION ) 
    {
     System.out.println("El numero de filas/columnas debe estar dentro del rango de " + MIN_DIMENSION + " a " + MAX_DIMENSION);    
     System.exit(1);
    }  
    
    //INICIALIZO LAS MATRICES, A Y B CON VALORES RANDOM, CS Y CH EN 0
    int [][] matrizA = inicializarMatriz(n, MIN_VALOR, MAX_VALOR);
    int [][] matrizB = inicializarMatriz(n, MIN_VALOR, MAX_VALOR);
    int [][] matrizCS = inicializarMatriz(n, 0, 0);
    int [][] matrizCH = inicializarMatriz(n, 0, 0);
    
    //CALCULO VALORES DE CS (SECUENCIAL)
    for(int i=0; i<n; i++)    
      calcularProducto(n, i, matrizA, matrizB, matrizCS);
    
    //CALCULO VALORES DE CH (CON CONCURRENCIA)
    //CREO UNA ESTRUCTURA PARA ALMACENAR LAS REFERENCIAS A LOS HILOS CREADOS    
    ArrayList<Hilo> hilos = new ArrayList<Hilo>();

    //CREO N HILOS
    for(int i=0; i<n; i++)  
      hilos.add(new Hilo(n, i, matrizA, matrizB, matrizCH));     
   
    //INICIO A LOS N HILOS 
    for (Hilo h: hilos) 
      h.start();          
           
    //ESPERO A LOS N HILOS 
    for (Hilo h: hilos) 
      h.join();
    
    //MUESTRO LOS RESULTADOS
    mostrarMatriz(n, matrizA,"A");    
    mostrarMatriz(n, matrizB,"B");
    mostrarMatriz(n, matrizCS,"CS"); 
    mostrarMatriz(n, matrizCH,"CH");
  }
}

Overwriting Matrices.java


In [91]:
!java Matrices.java 5


Matriz A
|    20     2   -23     9    14   |
|   -17    30   -30   -20    30   |
|   -17     6    32    29    -3   |
|    23     0    -7     8    29   |
|    25   -24    24    16    14   |

Matriz B
|    10     5    -7     4    -1   |
|    19    31    18   -22    17   |
|   -17    24    20     6    17   |
|    -5   -29   -24     9   -23   |
|    27   -13   -25    27   -16   |

Matriz CS
|   962  -833 -1130   357  -808   |
|  1820   315  -211  -278    -3   |
|  -826    67   246   172    44   |
|  1092  -662 -1218   905  -790   |
|  -316  -689  -861  1294  -617   |

Matriz CH
|   962  -833 -1130   357  -808   |
|  1820   315  -211  -278    -3   |
|  -826    67   246   172    44   |
|  1092  -662 -1218   905  -790   |
|  -316  -689  -861  1294  -617   |
